# Load Module

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

# Load Dataset

In [2]:
df_train = pd.read_csv('/kaggle/input/melting-point/train.csv')

# Data Overview

In [3]:
df_train.head()

,id,SMILES,Tm,Group 1,Group 2,Group 3,Group 4,Group 5,Group 6,Group 7,...,Group 415,Group 416,Group 417,Group 418,Group 419,Group 420,Group 421,Group 422,Group 423,Group 424
0,2175,FC1=C(F)C(F)(F)C1(F)F,213.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1222,c1ccc2c(c1)ccc3Nc4ccccc4c23,407.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2994,CCN1C(C)=Nc2ccccc12,324.15,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1704,CC#CC(=O)O,351.15,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2526,CCCCC(S)C,126.15,2,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df_train.info()
df_train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2662 entries, 0 to 2661
Columns: 427 entries, id to Group 424
dtypes: float64(1), int64(425), object(1)
memory usage: 8.7+ MB


,id,Tm,Group 1,Group 2,Group 3,Group 4,Group 5,Group 6,Group 7,Group 8,...,Group 415,Group 416,Group 417,Group 418,Group 419,Group 420,Group 421,Group 422,Group 423,Group 424
count,2662.000000,2662.000000,2662.000000,2662.000000,2662.000000,2662.000000,2662.000000,2662.000000,2662.000000,2662.000000,...,2662.000000,2662.000000,2662.0,2662.000000,2662.0,2662.0,2662.0,2662.0,2662.0,2662.0
mean,1652.887303,278.263452,1.020661,1.582269,0.114951,0.036439,0.036814,0.022915,0.013524,0.010143,...,0.001127,0.000751,0.0,0.001878,0.0,0.0,0.0,0.0,0.0,0.0
std,955.005512,85.117914,1.353640,3.738416,0.425964,0.208307,0.205516,0.164037,0.118732,0.117480,...,0.033558,0.027405,0.0,0.058126,0.0,0.0,0.0,0.0,0.0,0.0
min,1.000000,53.540000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25%,830.250000,217.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
50%,1642.500000,277.300000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
75%,2469.750000,325.150000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
max,3328.000000,897.150000,16.000000,42.000000,6.000000,3.000000,2.000000,2.000000,2.000000,2.000000,...,1.000000,1.000000,0.0,2.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
for x, y in df_train.isna().sum().items():
    if y != 0:
        print(x)

# Preparing the dataset

In [6]:
import tensorflow_decision_forests as tfdf
import tensorflow as tf

df_train = df_train.drop(['id','SMILES'], axis = 1)

def split_dataset(dataset, test_ratio=0.20):
    test_indices = np.random.rand(len(dataset)) < test_ratio
    return dataset[~test_indices], dataset[test_indices]
train_ds_pd, valid_ds_pd = split_dataset(df_train)

label = 'Tm'
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)
valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)

2025-09-17 03:45:32.039227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758080732.320608      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758080732.406464      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2025-09-17 03:45:52.179349: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


# Train Model

In [7]:
model = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.REGRESSION)
model.compile(metrics=['mse'])

model.fit(x=train_ds, validation_data=valid_ds)

model.evaluate(valid_ds)

Use /tmp/tmpzlk1pvhj as temporary training directory
Reading training dataset...
Training dataset read in 0:00:11.948696. Found 2112 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(550, shape=(), dtype=int32)
Validation dataset read in 0:00:07.241818. Found 550 examples.
Training model...


I0000 00:00:1758080772.202713      13 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1758080772.204000      13 kernel.cc:783] Collect training examples
I0000 00:00:1758080772.204038      13 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: NUMERICAL
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

I0000 00:00:1758080772.206643      13 kernel.cc:401] Number of batches: 3
I0000 00:00:1758080772.206697      13 kernel.cc:402] Number of examples: 2112
I0000 00:00:1758080772.217460      13 kernel.cc:802] Training dataset:
Number of records: 2112
Number of columns: 425

Number of columns by type:
	NUMERICAL: 425 (100%)

Columns:

NUMERICAL: 425 (100%)
	0: "Group_1" NUMERICAL mean:1.00663 min:0 max:16 sd:1.34275
	1: "Group_10" NUMERICAL mean:0.00094697 min:0 max:1 sd:0.0307583
	2:

Model trained in 0:00:21.085785
Compiling model...


I0000 00:00:1758080793.229567      13 decision_forest.cc:761] Model loaded with 300 root(s), 98024 node(s), and 235 input feature(s).
I0000 00:00:1758080793.232875      13 abstract_model.cc:1404] Engine "RandomForestOptPred" built


Model compiled.
1/1 [==============================] - 1s 1s/step - loss: 0.0000e+00 - mse: 3771.2546


[0.0, 3771.254638671875]

# Test Data Prediction

In [8]:
df_test = pd.read_csv('/kaggle/input/melting-point/test.csv')
df_test = df_test.drop(['id',"SMILES"], axis = 1)

test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(df_test, task=tfdf.keras.Task.REGRESSION)

df_submit = pd.read_csv('/kaggle/input/melting-point/sample_submission.csv')
df_submit['Tm'] = model.predict(test_ds)
df_submit.to_csv('/kaggle/working/sample.csv', index=False)

1/1 [==============================] - 1s 788ms/step
